In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import random
from random import seed
from random import sample

tensorflow.test.gpu_device_name()

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [37]:
title = "SWaT_Dataset_Normal_v0"
df = pd.read_csv("/content/drive/MyDrive/CI Project/" + title + ".csv")
df

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,22/12/2015 4:00:00 PM,2.470294,261.5804,2,2,1,244.3284,8.190080,306.1010,2.471278,...,1,1,10.02948,0.000000,4.277749,0.000256,1,1,1,Normal
1,22/12/2015 4:00:01 PM,2.457163,261.1879,2,2,1,244.3284,8.190080,306.1010,2.468587,...,1,1,10.02948,0.000000,4.277749,0.000256,1,1,1,Normal
2,22/12/2015 4:00:02 PM,2.439548,260.9131,2,2,1,244.3284,8.190080,306.1010,2.467305,...,1,1,10.02948,0.000000,4.277749,0.000256,1,1,1,Normal
3,22/12/2015 4:00:03 PM,2.428338,260.2850,2,2,1,244.3284,8.190080,306.1010,2.466536,...,1,1,10.02948,0.000000,4.277749,0.000256,1,1,1,Normal
4,22/12/2015 4:00:04 PM,2.424815,259.8925,2,2,1,244.4245,8.190080,306.1010,2.466536,...,1,1,10.02948,0.000000,4.277749,0.000256,1,1,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496795,28/12/2015 9:59:55 AM,2.460366,523.0430,2,2,1,262.0161,8.396437,328.5055,2.442316,...,2,1,250.81710,1.778105,189.855200,0.000128,1,1,1,Normal
496796,28/12/2015 9:59:56 AM,2.448836,522.9645,2,2,1,262.0161,8.396437,328.5055,2.442316,...,2,1,250.81710,1.778105,189.502700,0.000128,1,1,1,Normal
496797,28/12/2015 9:59:57 AM,2.434744,522.8860,2,2,1,262.0161,8.396437,328.6337,2.444879,...,2,1,250.81710,1.778105,189.502700,0.000128,1,1,1,Normal
496798,28/12/2015 9:59:58 AM,2.428338,522.9252,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.81710,1.649953,189.502700,0.000128,1,1,1,Normal


In [38]:
df.fillna(0, inplace=True)
df = df[15000:]
df = df.drop(columns=[' Timestamp'])
df = df.drop(columns=['Normal/Attack'])
df

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
15000,2.670147,651.2425,2,2,1,262.2404,8.386824,495.0269,2.471022,2,...,0.310604,2,1,253.4286,1.361612,192.6589,0.000128,1,1,1
15001,2.643564,651.7136,2,2,1,262.2404,8.386824,494.6680,2.471022,2,...,0.310604,2,1,253.4286,1.361612,192.6589,0.000128,1,1,1
15002,2.623387,651.9883,2,2,1,262.2404,8.386824,494.6680,2.471022,2,...,0.309067,2,1,253.4286,1.361612,192.6589,0.000128,1,1,1
15003,2.609935,652.3023,2,2,1,262.2404,8.386824,494.6680,2.471022,2,...,0.308619,2,1,253.4286,1.361612,192.6589,0.000128,1,1,1
15004,2.600327,652.4201,2,2,1,262.2404,8.388105,494.3860,2.471022,2,...,0.308619,2,1,253.4286,1.361612,192.6589,0.000128,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496795,2.460366,523.0430,2,2,1,262.0161,8.396437,328.5055,2.442316,2,...,0.308619,2,1,250.8171,1.778105,189.8552,0.000128,1,1,1
496796,2.448836,522.9645,2,2,1,262.0161,8.396437,328.5055,2.442316,2,...,0.308619,2,1,250.8171,1.778105,189.5027,0.000128,1,1,1
496797,2.434744,522.8860,2,2,1,262.0161,8.396437,328.6337,2.444879,2,...,0.308619,2,1,250.8171,1.778105,189.5027,0.000128,1,1,1
496798,2.428338,522.9252,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.308619,2,1,250.8171,1.649953,189.5027,0.000128,1,1,1


In [39]:
s1_df = df[['FIT101', 'LIT101', 'MV101', 'P101', 'P102']]
s2_df = df[['AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206']]
s3_df = df[['FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302']]
s4_df = df[['AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401']]
s5_df = df[['AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502', 'PIT503']]
s6_df = df[['FIT601', 'P601', 'P602', 'P603']]

In [40]:
def mergeDfs(df1, df2):
  return pd.merge(df1, df2, left_index=True, right_index=True)

In [41]:
def transformData(df):
    # Scale and transform the data
    for i in df.select_dtypes('object').columns:
        le = LabelEncoder().fit(df[i])
        df[i] = le.transform(df[i])

    x_scaler = MinMaxScaler()
    x_data = x_scaler.fit_transform(df)

    return x_data, x_scaler

In [42]:
def windowing_and_splitting(dataset, start, train, vali, window, horizon):
    # Window and split the data
    x_dataset = []
    y_dataset = []
    x_train = []
    y_train = []
    x_vali = []
    y_vali = []
    x_test = []
    y_test = []

    end = len(dataset)

    for i in range(start, end-5):
        indices = range(i, i+window)
        x_dataset.append(dataset[indices])
        indicey = range(i+window, i+1+window)
        y_dataset.append(dataset[i+window])

    temp = list(zip(x_dataset, y_dataset))
    random.shuffle(temp)
    res1, res2 = zip(*temp)

    for i in range(start, train):
        x_train.append(res1[i])
        y_train.append(res2[i])

    for i in range(train, vali):
        x_vali.append(res1[i])
        y_vali.append(res2[i])
    for i in range(vali, len(dataset)-5):
        x_test.append(res1[i])
        y_test.append(res2[i])

    return (
        np.array(x_train),
        np.array(y_train),
        np.array(x_vali),
        np.array(y_vali),
        np.array(x_test),
        np.array(y_test)
    )

In [43]:
hist_window = 5
horizon = 1
TRAIN_SPLIT = 400000
VALI_SPLIT = 450000

In [44]:
def trainModel(df, name):
  x_data, scaler = transformData(df)
  x_train, y_train, x_vali, y_vali, x_test, y_test, = windowing_and_splitting(x_data, 0, TRAIN_SPLIT, VALI_SPLIT, hist_window, horizon)

  model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(20, input_shape=(x_train.shape[1], x_train.shape[2])),
    tf.keras.layers.Dense(len(x_data[0]))
  ])
  model.compile(optimizer='adam', loss='mse')
  model.summary()
  print(model)

  model_path = 'sequential_7.h5'
  early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='min')
  checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
  callbacks = [early_stopings,checkpoint]
  history = model.fit(x_train, y_train,epochs=25,steps_per_epoch=100,validation_data=(x_vali, y_vali),validation_steps=50,verbose=0,callbacks=callbacks)

  # plt.figure(figsize=(16,9))
  # plt.plot(history.history['loss'])
  # plt.plot(history.history['val_loss'])
  # plt.title('Training Loss over Time')
  # plt.ylabel('Loss')
  # plt.xlabel('Epoch')
  # plt.legend(['Training Loss', 'Validation Loss'])
  # plt.show()

  # loss = model.evaluate(x_train, y_train)
  # print(f"Loss: {loss}")

  model.save("/content/drive/MyDrive/CI Project/Sequential_Model/Sequential_" + name + ".h5")
  return model

In [45]:
def getPredictions(df, model):
    x_data, scaler = transformData(df)
    x_normal = []
    y_normal = []

    for i in range(len(x_data) - 5):
        indices = range(i, i + 5)
        x_normal.append(x_data[indices])
        y_normal.append(x_data[i + 5])

    x_normal = np.array(x_normal)
    y_normal = np.array(y_normal)

    predictions_scaled = model.predict(x_normal)

    # Unscale the predictions
    predictions_unscaled = scaler.inverse_transform(predictions_scaled)

    return predictions_scaled, predictions_unscaled

## Predict for Stage 1 using Stage 6

In [46]:
s6_s1_model = trainModel(mergeDfs(s6_df, s1_df), "S6_S1_Model")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 20)                2400      
                                                                 
 dense_6 (Dense)             (None, 9)                 189       
                                                                 
Total params: 2,589
Trainable params: 2,589
Non-trainable params: 0
_________________________________________________________________


In [47]:
y_pred_6_1_s, y_pred_6_1_uns = getPredictions(mergeDfs(s6_df, s1_df), s6_s1_model)

15057/15057 [==============================] - 30s 2ms/step


In [48]:
y_pred_1_s = y_pred_6_1_s[:, -s1_df.shape[1]:]
y_pred_1_uns = y_pred_6_1_uns[:, -s1_df.shape[1]:]
s1_df_pred = pd.DataFrame(y_pred_1_uns, columns=['FIT101', 'LIT101', 'MV101', 'P101', 'P102'])
print(s1_df_pred)

          FIT101      LIT101     MV101      P101      P102
0       2.647047  657.731079  2.013331  2.012679  0.999855
1       2.633662  657.973938  2.011266  2.013643  0.999806
2       2.620375  658.177124  2.009252  2.014583  0.999717
3       2.604513  658.118713  2.006962  2.015467  0.999305
4       2.587280  657.957642  2.004722  2.016459  0.999013
...          ...         ...       ...       ...       ...
481790  2.507788  521.724426  1.992435  2.005877  1.001087
481791  2.496516  521.746643  1.990673  2.006745  1.001020
481792  2.486552  521.707336  1.989199  2.007463  1.000918
481793  2.476240  521.569885  1.987659  2.008186  1.000814
481794  2.467432  521.518921  1.986473  2.008806  1.000759

[481795 rows x 5 columns]


In [49]:
print(s1_df)

          FIT101    LIT101  MV101  P101  P102
15000   2.670147  651.2425      2     2     1
15001   2.643564  651.7136      2     2     1
15002   2.623387  651.9883      2     2     1
15003   2.609935  652.3023      2     2     1
15004   2.600327  652.4201      2     2     1
...          ...       ...    ...   ...   ...
496795  2.460366  523.0430      2     2     1
496796  2.448836  522.9645      2     2     1
496797  2.434744  522.8860      2     2     1
496798  2.428338  522.9252      2     2     1
496799  2.427057  522.8467      2     2     1

[481800 rows x 5 columns]


## Predict for Stage 2 using Stage 1 Predictions

In [50]:
s1_s2_model = trainModel(mergeDfs(s1_df, s2_df), "S1_S2_Model")

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 20)                2960      
                                                                 
 dense_7 (Dense)             (None, 16)                336       
                                                                 
Total params: 3,296
Trainable params: 3,296
Non-trainable params: 0
_________________________________________________________________


In [51]:
y_pred_1_2_s, y_pred_1_2_uns = getPredictions(mergeDfs(s1_df, s2_df), s1_s2_model)

15057/15057 [==============================] - 29s 2ms/step


In [52]:
y_pred_2_s = y_pred_1_2_s[:, -s2_df.shape[1]:]
y_pred_2_uns = y_pred_1_2_uns[:, -s2_df.shape[1]:]
s2_df_pred = pd.DataFrame(y_pred_2_uns, columns=['AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206'])
print(s2_df_pred)

            AIT201    AIT202      AIT203    FIT201     MV201      P201  \
0       262.265656  8.383146  493.352112  2.442235  1.976022  1.007102   
1       262.267212  8.383717  493.344788  2.443948  1.977840  1.007100   
2       262.273102  8.384047  493.353241  2.445778  1.980107  1.007156   
3       262.287048  8.384390  493.367859  2.448790  1.982927  1.007338   
4       262.292694  8.384470  493.312714  2.451748  1.985266  1.007218   
...            ...       ...         ...       ...       ...       ...   
481790  262.009216  8.389902  329.273590  2.463972  2.007522  0.998582   
481791  262.001617  8.390457  329.273315  2.465082  2.008448  0.998403   
481792  261.997528  8.390817  329.255096  2.466266  2.009464  0.998275   
481793  261.996979  8.391023  329.273254  2.467808  2.010832  0.998230   
481794  261.989990  8.391092  329.246338  2.469031  2.011433  0.997949   

            P202      P203      P204      P205      P206  
0       1.015137  1.997372  0.981636  0.976855  1.00

In [53]:
print(s2_df)

          AIT201    AIT202    AIT203    FIT201  MV201  P201  P202  P203  P204  \
15000   262.2404  8.386824  495.0269  2.471022      2     1     1     2     1   
15001   262.2404  8.386824  494.6680  2.471022      2     1     1     2     1   
15002   262.2404  8.386824  494.6680  2.471022      2     1     1     2     1   
15003   262.2404  8.386824  494.6680  2.471022      2     1     1     2     1   
15004   262.2404  8.388105  494.3860  2.471022      2     1     1     2     1   
...          ...       ...       ...       ...    ...   ...   ...   ...   ...   
496795  262.0161  8.396437  328.5055  2.442316      2     1     1     2     1   
496796  262.0161  8.396437  328.5055  2.442316      2     1     1     2     1   
496797  262.0161  8.396437  328.6337  2.444879      2     1     1     2     1   
496798  262.0161  8.396437  328.6337  2.445391      2     1     1     2     1   
496799  262.0161  8.396437  328.6337  2.445391      2     1     1     2     1   

        P205  P206  
15000 

## Predict for Stage 3 using Stage 2 Predictions

In [54]:
s2_s3_model = trainModel(mergeDfs(s2_df, s3_df), "S2_S3_Model")

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 20)                3200      
                                                                 
 dense_8 (Dense)             (None, 19)                399       
                                                                 
Total params: 3,599
Trainable params: 3,599
Non-trainable params: 0
_________________________________________________________________


In [55]:
y_pred_2_3_s, y_pred_2_3_uns = getPredictions(mergeDfs(s2_df, s3_df), s2_s3_model)

15057/15057 [==============================] - 30s 2ms/step


In [56]:
y_pred_3_s = y_pred_2_3_s[:, -s3_df.shape[1]:]
y_pred_3_uns = y_pred_2_3_uns[:, -s3_df.shape[1]:]
s3_df_pred = pd.DataFrame(y_pred_3_uns, columns=['FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302'])
print(s3_df_pred)

          FIT301      LIT301     MV301     MV302     MV303     MV304  \
0       2.137477  976.405334  0.998132  1.996634  1.008698  1.019896   
1       2.137943  976.349609  0.998115  1.997027  1.008355  1.019710   
2       2.137850  976.216248  0.998258  1.996956  1.008418  1.019807   
3       2.137715  976.167419  0.998296  1.996922  1.008392  1.019829   
4       2.137161  976.380249  0.998300  1.997056  1.008180  1.019695   
...          ...         ...       ...       ...       ...       ...   
481790  2.234783  956.012817  1.010507  1.989733  0.994407  1.000808   
481791  2.234973  956.061340  1.010750  1.989596  0.994666  1.000777   
481792  2.234541  956.135986  1.010818  1.989149  0.994805  1.001186   
481793  2.233931  956.254639  1.010688  1.988693  0.994891  1.001679   
481794  2.233319  956.430237  1.010602  1.988524  0.994900  1.001903   

            P301      P302  
0       0.996675  1.985621  
1       0.996596  1.985598  
2       0.996605  1.985413  
3       0.996598  1

In [57]:
print(s3_df)

          FIT301    LIT301  MV301  MV302  MV303  MV304  P301  P302
15000   2.043237  977.5538      1      2      1      1     1     2
15001   2.044903  977.6740      1      2      1      1     1     2
15002   2.044903  977.4738      1      2      1      1     1     2
15003   2.044903  977.6340      1      2      1      1     1     2
15004   2.044903  977.5538      1      2      1      1     1     2
...          ...       ...    ...    ...    ...    ...   ...   ...
496795  2.212087  955.8046      1      2      1      1     1     2
496796  2.210037  955.8847      1      2      1      1     1     2
496797  2.207731  955.9648      1      2      1      1     1     2
496798  2.206835  956.2051      1      2      1      1     1     2
496799  2.206835  956.1651      1      2      1      1     1     2

[481800 rows x 8 columns]


## Predict for Stage 4 using Stage 3 Predictions

In [58]:
s3_s4_model = trainModel(mergeDfs(s3_df, s4_df), "S3_S4_Model")

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 20)                3040      
                                                                 
 dense_9 (Dense)             (None, 17)                357       
                                                                 
Total params: 3,397
Trainable params: 3,397
Non-trainable params: 0
_________________________________________________________________


In [59]:
y_pred_3_4_s, y_pred_3_4_uns = getPredictions(mergeDfs(s3_df, s4_df), s3_s4_model)

15057/15057 [==============================] - 30s 2ms/step


In [60]:
y_pred_4_s = y_pred_3_4_s[:, -s4_df.shape[1]:]
y_pred_4_uns = y_pred_3_4_uns[:, -s4_df.shape[1]:]
s4_df_pred = pd.DataFrame(y_pred_4_uns, columns=['AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401'])
print(s4_df_pred)

            AIT401      AIT402    FIT401      LIT401      P401      P402  \
0         0.190816  198.068878  1.700811  821.770630  0.993643  2.016130   
1         0.123358  198.076294  1.700639  821.801575  0.993400  2.016140   
2         0.032207  198.083466  1.700534  821.666077  0.993216  2.016112   
3         0.045306  198.096741  1.700536  821.656677  0.993229  2.016148   
4        -0.124795  198.068909  1.701120  821.469238  0.992153  2.016436   
...            ...         ...       ...         ...       ...       ...   
481790  148.471786  156.127350  1.716856  942.546448  1.002525  2.001382   
481791  148.556808  156.083893  1.716162  942.554688  1.002240  2.001447   
481792  148.487534  156.046127  1.715670  942.541626  1.001675  2.001702   
481793  148.378540  156.026611  1.715337  942.577576  1.001229  2.001935   
481794  148.408661  156.025040  1.714349  942.569763  1.001773  2.001640   

            P403      P404     UV401  
0       0.989780  1.004831  2.002115  
1       0

In [61]:
print(s4_df)

         AIT401    AIT402    FIT401    LIT401  P401  P402  P403  P404  UV401
15000     0.000  197.1546  1.701855  827.8646     1     2     1     1      2
15001     0.000  197.1546  1.701855  827.6724     1     2     1     1      2
15002     0.000  197.1546  1.700061  828.1722     1     2     1     1      2
15003     0.000  197.2827  1.700061  828.1338     1     2     1     1      2
15004     0.000  197.2827  1.700061  828.2491     1     2     1     1      2
...         ...       ...       ...       ...   ...   ...   ...   ...    ...
496795  148.808  156.0882  1.715439  941.8354     1     2     1     1      2
496796  148.808  156.0882  1.715439  941.9893     1     2     1     1      2
496797  148.808  156.0882  1.715439  942.1815     1     2     1     1      2
496798  148.808  156.0882  1.713517  941.9508     1     2     1     1      2
496799  148.808  156.0882  1.713517  942.0662     1     2     1     1      2

[481800 rows x 9 columns]


## Predict for Stage 5 using Stage 4 Predictions

In [62]:
s4_s5_model = trainModel(mergeDfs(s4_df, s5_df), "S4_S5_Model")

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 20)                3440      
                                                                 
 dense_10 (Dense)            (None, 22)                462       
                                                                 
Total params: 3,902
Trainable params: 3,902
Non-trainable params: 0
_________________________________________________________________


In [63]:
y_pred_4_5_s, y_pred_4_5_uns = getPredictions(mergeDfs(s4_df, s5_df), s4_s5_model)

15057/15057 [==============================] - 30s 2ms/step


In [64]:
y_pred_5_s = y_pred_4_5_s[:, -s5_df.shape[1]:]
y_pred_5_uns = y_pred_4_5_uns[:, -s5_df.shape[1]:]
s5_df_pred = pd.DataFrame(y_pred_5_uns, columns=['AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502', 'PIT503'])
print(s5_df_pred)

          AIT501      AIT502      AIT503     AIT504    FIT501    FIT502  \
0       7.856138  182.557266  274.852753  12.099482  1.713413  1.273251   
1       7.856119  182.868637  274.864044  12.059145  1.713496  1.271713   
2       7.855679  182.999695  274.860077  12.033089  1.712936  1.270785   
3       7.855559  182.892685  274.898773  12.041352  1.712779  1.272070   
4       7.856019  183.008911  274.903564  12.040308  1.712968  1.278332   
...          ...         ...         ...        ...       ...       ...   
481790  7.878163  145.555206  263.892273  12.003837  1.726428  1.295010   
481791  7.877709  145.584274  263.887543  11.997691  1.726024  1.290990   
481792  7.877160  145.588470  263.846588  11.985331  1.725413  1.282604   
481793  7.877029  145.552826  263.876709  11.975919  1.724896  1.276444   
481794  7.877507  145.626633  263.880493  11.966024  1.724380  1.275483   

          FIT503    FIT504      P501      P502      PIT501    PIT502  \
0       0.740325  0.309291 

In [65]:
print(s5_df)

          AIT501    AIT502    AIT503    AIT504    FIT501    FIT502    FIT503  \
15000   7.857793  184.0041  274.4168  12.22763  1.710331  1.272576  0.739750   
15001   7.857793  184.0041  274.4168  12.22763  1.710331  1.271679  0.739750   
15002   7.857793  184.0041  274.4168  12.22763  1.710331  1.271679  0.739750   
15003   7.857793  184.0041  274.4168  12.22763  1.710331  1.278468  0.739750   
15004   7.857793  184.0041  274.5450  12.22763  1.710331  1.272320  0.739750   
...          ...       ...       ...       ...       ...       ...       ...   
496795  7.878621  145.2704  264.5475  12.03538  1.726352  1.292430  0.735269   
496796  7.878621  145.2704  264.5475  12.03538  1.724942  1.281158  0.735269   
496797  7.878621  145.2704  264.5475  12.03538  1.723789  1.272576  0.735269   
496798  7.878621  145.1166  264.5475  12.03538  1.723789  1.272576  0.735269   
496799  7.878621  145.1166  264.5475  12.03538  1.723789  1.279621  0.735269   

          FIT504  P501  P502    PIT501 

## Predict for Stage 6 using Stage 5 Predictions

In [66]:
s5_s6_model = trainModel(mergeDfs(s5_df, s6_df), "S5_S6_Model")

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 20)                3040      
                                                                 
 dense_11 (Dense)            (None, 17)                357       
                                                                 
Total params: 3,397
Trainable params: 3,397
Non-trainable params: 0
_________________________________________________________________


In [67]:
y_pred_5_6_s, y_pred_5_6_uns = getPredictions(mergeDfs(s5_df, s6_df), s5_s6_model)

15057/15057 [==============================] - 30s 2ms/step


In [68]:
y_pred_6_s = y_pred_5_6_s[:, -s6_df.shape[1]:]
y_pred_6_uns = y_pred_5_6_uns[:, -s6_df.shape[1]:]
s6_df_pred = pd.DataFrame(y_pred_6_uns, columns=['FIT601', 'P601', 'P602', 'P603'])
print(s6_df_pred)

          FIT601      P601      P602      P603
0      -0.017922  1.001363  1.010923  0.993738
1      -0.007592  1.002485  1.013672  0.996361
2      -0.003898  1.003321  1.015176  0.997549
3      -0.009615  1.003828  1.010271  0.995327
4      -0.015787  1.006667  0.996733  0.991307
...          ...       ...       ...       ...
481790 -0.001671  1.001071  1.005012  0.992431
481791 -0.009184  0.997947  1.001672  0.995191
481792 -0.011044  0.998483  1.001253  0.995491
481793 -0.008608  0.999149  0.999671  0.995247
481794  0.000929  1.001614  0.991554  0.994720

[481795 rows x 4 columns]


In [69]:
print(s6_df)

          FIT601  P601  P602  P603
15000   0.000128     1     1     1
15001   0.000128     1     1     1
15002   0.000128     1     1     1
15003   0.000128     1     1     1
15004   0.000128     1     1     1
...          ...   ...   ...   ...
496795  0.000128     1     1     1
496796  0.000128     1     1     1
496797  0.000128     1     1     1
496798  0.000128     1     1     1
496799  0.000128     1     1     1

[481800 rows x 4 columns]
